In [1]:
!pip install datasets transformers evaluate accelerate umap-learn umap-learn[plot]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 112.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8

In [2]:
from google.colab import drive
# mount drive to access data
drive.mount('/content/drive')
# load data folder into working directory
!cp -r drive/MyDrive/data .
!cp -r drive/MyDrive/bert_emotion .

Mounted at /content/drive


In [37]:

from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
 AdamW, TrainingArguments, Trainer
import numpy as np
import evaluate
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import tqdm

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [5]:
# loading data with preprocessed tweets
df = pd.read_csv('data/cleaned_tweets.csv')

In [23]:
# drop other columns
df = df['TweetText']
dates = df['Timestamp']

### Prepare Input for BERT

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [9]:
def return_dataset(df, select=None):

  dataset = Dataset.from_pandas(df)
  if select is not None:
    dataset = dataset.select(range(select))

  dataset = dataset.rename_column("TweetText", "text")

  dataset = dataset.map(tokenize_function, batched=True)

  dataset.set_format("torch")

  return dataset

In [10]:
dataset = return_dataset(df)

Map:   0%|          | 0/561730 [00:00<?, ? examples/s]

Load our pretrained model.

In [54]:
model = AutoModelForSequenceClassification.from_pretrained("bert_emotion")

In [55]:
for param in model.parameters():
  param.requires_grad = False

In [65]:
loader = DataLoader(dataset, \
           batch_size=128, \
           pin_memory=False, \
           shuffle=False, \
           num_workers=4)

In [66]:
model = model.to('cuda')

In [67]:
model.train()
predictions = torch.empty(0,).to(device)

for inputs in tqdm.tqdm(loader):

  with torch.no_grad():

    outputs = model(inputs['input_ids'].to(device))
    outputs = torch.sigmoid(outputs.logits)
    outputs = torch.round(outputs)
    predictions = torch.cat((predictions,outputs))

  0%|          | 0/4389 [00:00<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 4>:8                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/bert/modeling_bert.py:1562 in        │
│ forward                                                                                          │
│                                                                                                  │
│   1559 │   │   """                                                                               │
│   1560 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return  │
│   1561 │   │                                                                                     │
│ ❱ 1562 │   │   outputs = self.bert(                                                              │
│   1563 │   │   │   input_ids,                                                                    │
│   1564 │   │   │   attention_mask=attention_mask,                                                │
│   1565 │   │   │   token_type_ids=token_type_ids,                                                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/bert/modeling_bert.py:1020 in        │
│ forward                                                                                          │
│                                                                                                  │
│   1017 │   │   │   inputs_embeds=inputs_embeds,                                                  │
│   1018 │   │   │   past_key_values_length=past_key_values_length,                                │
│   1019 │   │   )                                           